<a href="https://colab.research.google.com/github/acoiman/swb/blob/main/surface_water_balance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Estimación el Balance de Agua Superficial y el Volumen de Agua de las Cuencas en Sur América (WWF HydroSHEDS Nivel 2).

A través de esta notebook vamos a usa a hacer el cálculo aproximado del Balance de Agua Superficial (BAS) y el Volumen de Agua (VA) en las cuencas de sur américa correspondientes al nivel 2 de conjunto de datos WWF HydroSHEDS. Este análisis lo vamos a efectuar en una serie temporal mensual entre enero del 2000 y diciembre del 2019. 

Para lograr este objetivo vamos a usar la API Python de Google Earh Engine (GEE) y otras librerías Python. El conjunto de datos que vamos a emplear es el siguiente:

- Cuencas nivel 2  de [WWF HydroSHEDS](https://developers.google.com/earth-engine/datasets/catalog/WWF_HydroSHEDS_v1_Basins_hybas_2)
- [GLDAS-2.1: Global Land Data Assimilation System](https://developers.google.com/earth-engine/datasets/catalog/NASA_GLDAS_V021_NOAH_G025_T3H). De los datos  GLDAS-2.1  vamos a selecionar la proporción total de precipitación (`Rainf_f_tavg`), la evapotranspiración (`Evap_tavg`) y la escorrentía superficial de la lluvia (`Qs_acc`). La resolución espacial de estos datos es de 0.25 grados de arco, aproximadamente 30 Km.

El cálculo del BAS es el siguiente:

$${BAS(mm/mes)} = {{precipitación} - {evapotranspiración} - {escorrentía}}.$$

Es una primera aproximación al cálculo del volumen de agua en la cuenca porque no se tomaron otros factores como el suelo y las aguas subterráneas.

El cálculo del VA es el siguiente: 

$${VA(m^3/mes)} = {{BS(mm/mes)} * 1000 * {Area Cuenca(m^2)}}.$$

Estos datos se cruzarán con otros datos de puntos de calor y área quemadas por cuenca con el fin de establecer patrones entre los incendios forestales y la disponibilidad de agua.

Para iniciar procedamos a instalar algunos paquetes necesarios.

In [ ]:
# installl packages
!pip install geopandas
!pip install eeconvert
!pip install geemap
!pip install arrow

Accedemos a los servicios de GEE.

In [ ]:
# Authenticate to Earth Engine
!earthengine authenticate

Instructions for updating:
non-resource variables are not supported in the long term
Running command using Cloud API.  Set --no-use_cloud_api to go back to using the API

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=8-HMnuJDEDOr6ihS1rVCbdh4q19bFzcsSkb7ACfhB4U&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g4rv3FQWPdUJkm4bR4DjmQV0xkd04EuMVObUG9QSBpvaqpyU9s-fzU

Successfully saved authorization token

Procedamos a importar algunos paquetes necesarios.

In [ ]:
# import packages
from IPython.display import Image 
import matplotlib.pyplot as plt
from pprint import  pprint
import eeconvert
import ee
import folium
import geemap
import geopandas
from datetime import date
from datetime import datetime
import arrow
import pandas as pd

Inicialicemos GEE. 

In [ ]:
# initialize GEE
ee.Initialize()

Accedamos a los servicios de almacenamiento de Google Drive.

In [ ]:
# Authenticate to Google Drive
# Mount Google Drive
from google.colab import drive # import drive from google colab

# default location for the drive
ROOT = "/content/drive" 
print(ROOT) # print content of ROOT (optional)

drive.mount(ROOT) # we mount the google drive at /content/drive

/content/drive
Mounted at /content/drive


Cambiémonos a nuestra carpeta de trabajo

In [ ]:
%cd "drive/MyDrive/Colab_Notebooks/Taller_GEE_Inc_2021"

/content/drive/MyDrive/Colab_Notebooks/Taller_GEE_Inc_2021


Llamemos a la colección **WWF HydroSHEDS Basins nivel 2***

In [ ]:
# HydroSHEDS basins level 2
basin_l2 = ee.FeatureCollection("WWF/HydroSHEDS/v1/Basins/hybas_2")

Crear una lista con el ID de las cuencas Suramérica nivel 2

In [ ]:
mainBasiIDs = [6020029280, 6020017370, 6020021870, 6020008320,  6020014330, 6020000010, 6020006540]

Seleccionaremos ahora las cuencas Suramérica de la colección  HydroSHEDS

In [ ]:
Basin00 = basin_l2.filter(ee.Filter.eq("HYBAS_ID", mainBasiIDs[0]))
Basin01 = basin_l2.filter(ee.Filter.eq("HYBAS_ID", mainBasiIDs[1]))
Basin02 = basin_l2.filter(ee.Filter.eq("HYBAS_ID", mainBasiIDs[2]))
Basin03 = basin_l2.filter(ee.Filter.eq("HYBAS_ID", mainBasiIDs[3]))
Basin04 = basin_l2.filter(ee.Filter.eq("HYBAS_ID", mainBasiIDs[4]))
Basin05 = basin_l2.filter(ee.Filter.eq("HYBAS_ID", mainBasiIDs[5]))
Basin06 = basin_l2.filter(ee.Filter.eq("HYBAS_ID", mainBasiIDs[6]))

Procedamos a unir todas las cuencas.

In [ ]:
saBasins = Basin00.merge(Basin01)\
                .merge(Basin02).merge(Basin03)\
                .merge(Basin04).merge(Basin05).merge(Basin06)

Disolvamos las líneas entre cuencas para obtener el polígono de Suramérica.

In [ ]:
southAmer = saBasins.union()

Ahora visualicemos el polígono de Suramérica y las cuencas Suramérica en un mapa.

In [ ]:
# create map
Map = geemap.Map(center=(-9.84, -62.45), zoom=3)

# add layers to map
Map.addLayer(southAmer, {}, 'Sur América', False)
Map.addLayer(saBasins, {}, 'Cuenca Nivel 2', True)
Map.addLayerControl()
Map

Calculemos las fechas del primer día del mes entre un periodo de tiempo determinado.

In [ ]:
def ini_dates(start, end):
    
    '''function to create a pandas series containing the first day of a month for given period
    Argumets:
    start: initial date in format 'year-month-day'
    end: final date in format 'year-month-day'
    
    return: a pandas Series with dates in format 'year-month-day'
        
    Example: if you want to get the first day between January and February you should enter 
    '2000-01-01' as initial date and '2000-03-01' as final date
    '''
    range= pd.date_range(start,end , freq='1M')-pd.offsets.MonthBegin(1)
    serie = pd.Series(range.format())
    
    return serie

In [ ]:
startDates = ini_dates('2000-01-01','2020-01-01')

In [ ]:
startDates

0      2000-01-01
1      2000-02-01
2      2000-03-01
3      2000-04-01
4      2000-05-01
          ...    
235    2019-08-01
236    2019-09-01
237    2019-10-01
238    2019-11-01
239    2019-12-01
Length: 240, dtype: object

Calculemos las fechas del último día del mes entre un periodo de tiempo determinado.

In [ ]:
def fin_dates(start, end):
    
    '''function to create a pandas series containing the last day of a month for given period
    Argumets:
    start: initial date in format 'year-month-day'
    end: final date in format 'year-month-day'
    
    return: a pandas Series with dates in format 'year-month-day'
        
    Example: if you want to get the last day between January and February you should enter 
    '2000-01-01' as initial date and '2000-03-01' as final date
    '''
    range= pd.date_range(start,end , freq='1M')-pd.offsets.MonthEnd(0)
    serie = pd.Series(range.format())
    
    return serie

In [ ]:
endDates = fin_dates('2000-01-01','2020-01-01')

In [ ]:
endDates

0      2000-01-31
1      2000-02-29
2      2000-03-31
3      2000-04-30
4      2000-05-31
          ...    
235    2019-08-31
236    2019-09-30
237    2019-10-31
238    2019-11-30
239    2019-12-31
Length: 240, dtype: object

En esta sección vamos a calcular Balance de Agua Superficial y el Volumen de Agua por mes para el período y área de estudio. Luego calcularemos las estadísticas zonales (media) del BAS para cada cuenca. Convertiremos el FeatureCollection a un GeoDataFrame para calcular el área por cuenca y el volumen de agua por cuenca. Luego se almacenará toda esta serie temporal en una lista.

In [ ]:
# list to store dataframe values
lst = []
 
for d1, d2 in zip(startDates, endDates):
    # d1: start date
    # d2: end date

    # calculate number of days between d1 and d2
    a = arrow.get(d1)
    b = arrow.get(d2)
    delta = (b-a)
    numdays = delta.days

    # GLDAS-2.1: Global Land Data Assimilation System
    # https://developers.google.com/earth-engine/datasets/catalog/NASA_GLDAS_V021_NOAH_G025_T3H
    dataset = ee.ImageCollection('NASA/GLDAS/V021/NOAH/G025/T3H').filter(ee.Filter.date(d1, d2)) 
    # select total precipitation rate and calculate the mean
    precip_rate = dataset.select('Rainf_f_tavg').mean()
    # clip to all basins
    precip_clip = precip_rate.clip(southAmer)
    # convert units to get monthly data
    # precipitation is in kg m^2 s^-1  accumulated over 3 hour interval 
    # Rainf_f_tavg (month){kg/m2} =
    # Rainf_f_tavg (month){kg/m2/sec} * 10800{sec/3hr} * 8{3hr/day} * 30{days}
    precip = precip_clip.multiply(10800).multiply(8).multiply(numdays)
   
    # select evapotranspiration and calculate the mean
    evap = dataset.select('Evap_tavg').mean()
    # clip to all basins
    evap_clip =  evap.clip(southAmer)  
    # convert units to get monthly data
    # evapotraspiration is in kg m^2 s^-1  accumulated over 3 hour interval 
    # Evap_tavg (month){kg/m2} =
    # Evap_tavg (month){kg/m2/sec} * 10800{sec/3hr} * 8{3hr/day} * 30{days}
    ET =  evap_clip.multiply(10800).multiply(8).multiply(numdays) 
   
    # select storm surface runoff and calculate the mean
    ssr = dataset.select('Qs_acc').mean()
    # clip to all basins
    ssr_clip  =  ssr.clip(southAmer)
    # convert units to get monthly data
    # runoff is in kg m^2  accumulated over 3 hour interval 
    # Qs_acc (month){kg/m2} = Qs_acc (month){kg/m2/3hr} * 8{3hr/day} * 30{days}
    runoff =  ssr_clip.multiply(8).multiply(30)
   
    # calculate Surface Water Balance
    SWB= precip.subtract(ET);
    WB = SWB.subtract(runoff).rename('WB');
   
    # add reducer output to the features in the collection.
    # add the mean of  Surface Water Balance to each basin
    saBasinsWB = WB.reduceRegions(**{
        'collection': saBasins,
        'reducer': ee.Reducer.mean(),
        'scale': 30000
    })
    # conver fc to gdf
    df = eeconvert.fcToGdf(saBasinsWB)

    # create FECHA column
    df['FECHA'] =  d1.replace('-', '')[:-2]
   
    # reproject to South America Albers Equal Area Conic
    df_proj= df.to_crs("+proj=aea +lat_1=-5 +lat_2=-42 +lat_0=-32 +lon_0=-60 +x_0=0 +y_0=0 +ellps=aust_SA +units=m +no_defs")
   
    # calculate area in m^2
    df_proj["AREA_M2"] = df_proj['geometry'].area

    # calculate water volume in m^3
    df_proj["VOLUMEN_AGUA_M3"] =  (df_proj['mean'] / 1000) * df_proj['AREA_M2']
   
    # select and rename columns
    df_proj = df_proj[['HYBAS_ID', 'mean', 'FECHA', 'VOLUMEN_AGUA_M3']]
    df_proj = df_proj.rename(columns={'mean': 'WB'})

    # reorder columns
    df_final = df_proj[['FECHA', 'HYBAS_ID','WB', 'VOLUMEN_AGUA_M3']]
   
    # df values to list
    dflist = df_final.values.tolist()
    # append dflist to another list
    lst.append(dflist)

/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


Eliminemos los paréntesis adicionales de la lista.

In [ ]:
# flatten list
final_list = [i for sublist in lst for i in sublist]

A partir de la lista anterior vamos a crear una DataFrame, luego los visualizaremos. 

In [ ]:
# create  new dataframe from final_list
df2 = pd.DataFrame(final_list, columns =['FECHA', 'HYBAS_ID','WB', 'VOLUMEN_AGUA_M3'])

In [ ]:
# visualize new dataframe
df2.head()

,FECHA,HYBAS_ID,WB,VOLUMEN_AGUA_M3
0,200301,6020029280,38.400772,1.953174e+10
1,200301,6020017370,-23.417259,-3.544312e+10
2,200301,6020021870,12.810889,1.387266e+10
3,200301,6020008320,85.343679,1.790697e+11
4,200301,6020014330,63.349821,2.282604e+11


Verifiquemos la consistencia de los datos

In [ ]:
len(df2)==(len(startDates)*7)

True

Finalmente exportemos los datos en formato csv

In [ ]:
df2.to_csv('wb2000_2019.csv', index=False)